# Preface

The following examples are what I wrote when learning shell. Some are typical. Refresh it when necessary.

# 模式匹配和位置参数

$#：参数个数

$*：将所有命令行视为单个字符串（分隔以$IFS的第一个字符为准，一般为空格，tab，换行）

$@视为单独字符串（适合保留单个参数中的空白部分）

双引号会确切地处理括起来文字中的转义字符和变量、算术、命令替换等；而单引号都看作字面上的意义!

In [9]:
path=/home/cshi/long.file.name
echo $path
echo ${path#/*/} #从开头匹配最短的，返回剩下的
echo ${path##/*/} #从开头匹配最长的，返回剩下的
echo ${path%.*} #从结尾匹配最短的，返回剩下的
echo ${path%%.*} #从结尾匹配最长的，返回剩下的
echo ${#path} #返回字符长度

# 如果给定参数一，且非空，则取其值，否则取/dev/tty.诸如类似的还可将-换成 =,+,?
filename=${1:-/dev/tty}

set -- hello "hi there" world #设置位置参数
echo there are $# total arguments

echo '$*-------------'
for i in $*; do echo i is $i;done
echo '$@-------------'
for i in $@;do echo i is $i;done  # 没有双引号时，$*与$@一样
echo 'quoted $*-------------'
for i in "$*";do echo i is $i;done #加了引号后，$*表示一个字符串
echo 'quoted *-------------'
for i in "$@";do echo i is $i;done #加了引号后，$@保留了空白，此种方式用得多

/home/cshi/long.file.name
cshi/long.file.name
long.file.name
/home/cshi/long.file
/home/cshi/long
25
there are 3 total arguments
bash: there: command not found
$*-------------
i is hi
i is there
i is world
$@-------------
i is hi
i is there
i is world
quoted $*-------------
i is hi there world
quoted *-------------
i is hi there
i is world


# []和test

if [ -f "$file" ]：为一般文件
-s file：文件非空

-n string：字符串非null
-z string：字符串为空
n1 -lt n2：n1小于等于n2

if [ -f "$file" ] && ! [ -w "$file" ]：如果是文件且不可写（一定要加引号）

if [ "X$1" =  "X-f" ]; then :    为了避免-号干扰到test命令，前面加了前置字母，且加个:号表示不做任何事

if [ $1 = "create" ]; then，如果这样写，要注意一旦$1为空，则表达式错误，报错：
> [: =: unary operator expected

# 判断、循环与函数

## if

In [27]:
 # !写在前面
if ! grep 'cshi' /etc/passwd; then #如果有必要，则重定位标准输出和错误
     echo "not found"
fi
if ! grep 'not-exist' /etc/passwd; then
    echo "not found"
fi

a='a'
b='b'
if [ $a=$b ]; then echo equal;fi
if [ $a = $b ]; then echo equal2;fi #一定要加空格！！！是比较不是赋值

cshi:x:1002:1002::/home/cshi:/bin/bash
not found
equal


## for/while/until

In [ ]:
# 计算从1加到100
i=1  #等号左右不能加空格！！不能再像c、python一样加空格了！
sum=0

while [ "$i" -le 10 ]
do
 sum=$(($sum+$i)) 
 i=$((++i)) #shell算术运算符都位于$(())内
done

echo $sum


In [ ]:
# 行读取文件
printf "*************************************\n"
echo " cat file while read line"
cat test.txt | while read line
do
  echo $line;
done

printf "*************************************\n"
echo "while read line <file"
while read line
do
  echo $line;
done < test.txt

printf "*************************************\n"
echo "for line in cat test.txt"
SAVEIFS=$IFS
IFS=$(echo -en "\n")
for line in $(cat test.txt)
do
  echo  $line;
done
IFS=$SAVEIFS
#当文件中有空格或者tab 时，一定要设置一下IFS变量

## case

> 结构：

```
case $1 in
-f)
    ...
    ;;
*)
    ;;
esac
```

## function

In [ ]:
# 每30秒检查特定用户是否登陆
check() {
    while true; do 
        if who | grep cshi > /dev/null; then 
            break;
        fi
        sleep 30
    done
    echo "done"
}

wait_for_user() {
     until who | grep cshi >/dev/null #until的条件为假时，继续执行
     do
      sleep ${2:-30}
     done
 }
 
 #父脚本参数会暂时被函数参数所隐藏，即$1,$2指的是函数参数，$0仍为脚本名
 #wait_for_user cshi 3 

# grep

In [7]:
# 后向引用，最多可包括9
echo 'ababcd' | grep '\(ab\).\1'
echo 'ababcd' | grep '\(ab\).*\1'

echo abbba | grep 'ab\{4,\}' #至少出现几次


ababcd


: 1

In [ ]:
#去掉空行，注意不适合dos格式的文件，含有^M
grep -v '^$' a.txt > tmp 

# Sed

> 无-n有-p时，打印所有行，且匹配行会多打印
有n无p时，不打印
有n有p时，仅打印匹配行，这是最为常用的！！

In [ ]:
sed -n '$p' a.txt  #打印最后一行
sed -n '1,10p' a.txt #打印1-10行
sed -n '2,$p' a.txt >tmp #相当于删除民第一行
sed '/foo/,/bar/ s/a/A/g' a.txt #从含有foo的行开始到含有bar的行结束，a换成A
sed '/bc/!s/d/D/' a.txt #将没有bc的行的所有d换成D，即!放在正则后面表示否定
sed -i 's/s/S/' #直接将文件a中的s换成S!!!

In [ ]:
#unix转dos
sed 's/$/\r/' unix.txt >dos.txt

#dos转unix
sed 's/.$//' dos.txt >unix.txt 

In [15]:
cat << eof >tmp
my name is cshi
yes i can
eof
#cat tmp
echo "## 打印匹配行与行号"
sed -n -e '/my/p' -e '/my/=' tmp

echo 'abcd dcba' | sed 's/a.*d/A/' #默认匹配最长的串
echo 'abcd dcba' | sed 's/a*d/A/' #可以匹配0个a
echo 'abcd dcba' | sed 's/a*d/A/g'

echo abc | sed 's/b*/B/'
echo abc | sed 's/b*/B/g'

#后向引用
echo /home/cshi/ | sed 's;\(/home\)cshi/;\1/cs/;' #以;作为分隔符

# 打印含有/home的帐号
cat /etc/passwd | sed -n '/\/home/p' # 要转义/

## 打印匹配行与行号
my name is cshi
1
Acba
abcA dcba
abcA Acba
Babc
BaBcB
/home/cshi/
oprofile:x:16:16:Special user account to be used by OProfile:/home/oprofile:/sbin/nologin
iqiyioops:x:1001:1001:iqiyioops Puppet-managed User:/home/iqiyioops:/bin/bash
cshi:x:1002:1002::/home/cshi:/bin/bash
test:x:1003:1003::/home/test:/bin/bash


# cut

In [16]:
# -d表示分隔符，默认\t，-f表示字段
cut -d: -f 1,6 /etc/passwd | tail 

grafana:/usr/share/grafana
logstash:/opt/logstash
sphinx:/var/lib/sphinx
redis:/var/lib/redis
hacluster:/var/lib/heartbeat/cores/hacluster
ricci:/var/lib/ricci
nrpe:/var/run/nrpe
telegraf:/etc/telegraf
test:/home/test
cloud-agent:/opt/cloud-agent/agent


# sort

In [20]:
# t限定分隔符(注意与cut分隔符的区别)，k指定字段
sort -t: -k1,1 -r /etc/passwd | tail

echo "## sorted by number"
# n 以数值排，r反序
sort -t: -k3nr /etc/passwd | tail

dbus:x:81:81:System message bus:/:/sbin/nologin
daemon:x:2:2:daemon:/sbin:/sbin/nologin
cshi:x:1002:1002::/home/cshi:/bin/bash
couchbase:x:496:496:Couchbase system user:/opt/couchbase:/sbin/nologin
cloud-agent:x:850:850:Cloud-Agent agent:/opt/cloud-agent/agent:/bin/bash
bitlbee:x:489:488:BitlBee User:/var/lib/bitlbee:/sbin/nologin
bin:x:1:1:bin:/bin:/sbin/nologin
apache:x:48:48:Apache:/var/www:/sbin/nologin
adm:x:3:4:adm:/var/adm:/sbin/nologin
abrt:x:173:173::/etc/abrt:/sbin/nologin
## sorted by number
uucp:x:10:14:uucp:/var/spool/uucp:/sbin/nologin
mail:x:8:12:mail:/var/spool/mail:/sbin/nologin
halt:x:7:0:halt:/sbin:/sbin/halt
shutdown:x:6:0:shutdown:/sbin:/sbin/shutdown
sync:x:5:0:sync:/sbin:/bin/sync
lp:x:4:7:lp:/var/spool/lpd:/sbin/nologin
adm:x:3:4:adm:/var/adm:/sbin/nologin
daemon:x:2:2:daemon:/sbin:/sbin/nologin
bin:x:1:1:bin:/bin:/sbin/nologin
root:x:0:0:root:/root:/bin/bash


# awk

In [1]:
# 默认以空白分隔字段，且不要忘记中间的逗号，否则会连接相信的所有值！  输出以空格为分界
awk -F: '{print "user",$1,"is readlly",$5}' /etc/passwd | tail

user grafana is readlly grafana user
user logstash is readlly logstash
user sphinx is readlly Sphinx Search
user redis is readlly Redis Server
user hacluster is readlly heartbeat user
user ricci is readlly ricci daemon user
user nrpe is readlly NRPE user for the NRPE service
user telegraf is readlly 
user test is readlly 
user cloud-agent is readlly Cloud-Agent agent


# 数制处理

In [14]:
echo "obase=8;10" | bc
echo "obase=16;255" | bc
let num=012 #八进制
echo $num
let num=3#12 # 3进制
echo "$num"

((num=012))
((num2=8#12))
((num3=0xff))
((num4=16#ff))
((num5=2#00001111))
echo "(()):$num,$num2,$num3,$num4,$num5"

12
FF
10
5
(()):10,10,255,255,15


# subShell和代码块

subshell:以()包裹，在另外的进程中执行，不改变主脚本目录
tar –cf . | (cd /newdir; tar –xpf .)
 
代码块：以{}包裹，不会建立新进程，且会对主脚本状态造成影响。{}在shell中为关键字，所以}的位置要单独成行


# misc

In [15]:
echo i love you | wc -l
echo i love you | wc -c
echo i love you | wc -w

#将当前文件夹内所有文件的名字中得所有空格替换为_。其中g代表所有，
#rename 's/ /_/g' *  

# ubuntu脚本apt-get安装
#echo 'e' | sudo -S apt-get -y install $i

#改变read的分隔符且不影响循环内部的IFS
while IFS=: read user pass uid gid fullname homedir shell; do
    echo $user,$homedir; 
done < /etc/passwd

exec 2> /tmp/$0.log     #重定向shell本身的标准错误
exec 3 < /tmpfile        #打开新文件描述符3
read name < &3            #从3指向的文件读取



1
11
3
root,/root
bin,/bin
daemon,/sbin
adm,/var/adm
lp,/var/spool/lpd
sync,/sbin
shutdown,/sbin
halt,/sbin
mail,/var/spool/mail
uucp,/var/spool/uucp
operator,/root
games,/usr/games
gopher,/var/gopher
ftp,/var/ftp
nobody,/
dbus,/
vcsa,/dev
rpc,/var/cache/rpcbind
abrt,/etc/abrt
apache,/var/www
haldaemon,/
ntp,/etc/ntp
saslauth,/var/empty/saslauth
postfix,/var/spool/postfix
tomcat,/usr/share/tomcat6
webalizer,/var/www/usage
sshd,/var/empty/sshd
mysql,/var/lib/mysql
tcpdump,/
oprofile,/home/oprofile
puppet,/var/lib/puppet
iqiyioops,/home/iqiyioops
couchbase,/opt/couchbase
nagios,/var/spool/nagios
exim,/var/spool/exim
cshi,/home/cshi
rabbitmq,/var/lib/rabbitmq
nginx,/var/lib/nginx
memcached,/var/run/memcached
dockerroot,/var/lib/docker
rpcuser,/var/lib/nfs
nfsnobody,/var/lib/nfs
bitlbee,/var/lib/bitlbee
influxdb,/var/lib/influxdb
zabbix,/var/lib/zabbix
grafana,/usr/share/grafana
logstash,/opt/logstash
sphinx,/var/lib/sphinx
redis,/var/lib/redis
hacluster,/var/lib/heartbeat/cores/hacluster
